In [46]:
from bs4 import BeautifulSoup
import pandas as pd
import secret
import re
import pyscreenshot as ImageGrab
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from bs4 import Comment
import time

In [47]:
url = "https://www.sexoffender.go.kr/m1s2_2.nsc"
driver = webdriver.Chrome('chromedriver.exe')


In [48]:
def click(btn):
    action = ActionChains(driver)
    action.click(on_element = btn)
    action.perform()

In [49]:
driver.get(url)

In [50]:
agree_btn = driver.find_element_by_id('agree_btn') # 동의함 버튼 클릭
click(agree_btn)

In [51]:
driver.find_element_by_id('select04').click() # 주민등록번호 인증 클릭

In [52]:
name_tag = driver.find_element_by_id('kname') # 이름 입력
actions = webdriver.ActionChains(driver).send_keys_to_element(name_tag).send_keys('김명수')
actions.perform()

In [53]:
p1_tag = driver.find_element_by_id('socno1') # 주민등록번호 입력
p2_tag = driver.find_element_by_id('socno2')
action1 = webdriver.ActionChains(driver).send_keys_to_element(p1_tag).send_keys(secret.p1)
action2 = webdriver.ActionChains(driver).send_keys_to_element(p2_tag).send_keys(secret.p2)

action1.perform()
action2.perform()

In [54]:
iden_btn = driver.find_element_by_id('iden_auth_btn') # 인증 버튼 클릭
click(iden_btn)

In [58]:
try:
    action = webdriver.ActionChains(driver).send_keys(Keys.ENTER)
    action.perform()
except:
    pass

In [60]:
driver.get('https://www.sexoffender.go.kr/m1s2_3.nsc') #조건 검색으로 브라우저 이동

In [61]:
driver.find_element_by_id('condition04').click() # 주민등록번호 인증 클릭

In [62]:
select = Select(driver.find_element_by_id('city1'))
select.select_by_value('서울특별시') #서울특별시 선택

search_btn = driver.find_element_by_xpath('//*[@id="search_city"]/a') #검색 버튼 클릭
click(search_btn)

In [63]:
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

In [64]:
hand = soup.find_all('a')

In [65]:
for i in hand:
    print(i.text)

본인인증삭제
화면축소
화면확대
초기화
성범죄자 알림e
성범죄자 찾아보기
지도로 검색
조건으로 검색
정보통신망 고지
공개 및 고지 정보의 정정
정정 신청 안내
정정 신청
정정 신청 처리 결과 확인
공개현황(실시간)
사용안내
열람방법 및 절차
홈페이지 이용안내
프로그램 설치 안내
시스템 오류시 조치방법
문의사항 안내
제도안내
성범죄자 신상공개
성범죄자 신상정보 모바일고지
성범죄자 신상정보 등록제도
취업제한제도(점검결과공개)
재범방지 교육제도
포상금 제도
자주묻는 질문
피해예방 지원
성폭력 예방교육
성범죄 예방 및피해 대응요령
피해자 지원
관련법령
관련기관
안전정보
긴급 안심서비스
안전 귀가 지원서비스
지도로 검색
조건으로 검색
정보통신망고지
공개 및 고지 정보의 정정
정정 신청 안내
정정 신청
정정 신청 처리결과 확인
공개현황 (실시간)
Home
성범죄자 찾아보기
검색
전체
어린이집
유치원
초등학교
중학교
고등학교
검색
검색
선호상

															   		서울특별시 송파구 백제고분로46길 7-14
													 
													

														 
															  
															  
															   
															  
															  
															   		서울특별시 송파구 백제고분로46길 7-14
															  
														 
													
이창민

															   		서울특별시 강북구 노해로27길 96
													 
													

														 
															  
															  
															   
															  
															  
															   		서울특별시 강북구 노해로27길 96
															  
														 


In [66]:
def get_image(soup):
    cnt = 1
    for i in soup.select('tr td a'):
        i = i.text.strip()
        if cnt % 3 == 1:
            driver.find_element_by_link_text(f"{i}").click()
            time.sleep(20)
            im = ImageGrab.grab(bbox=(545,180,703,375)) 
            im.save(f'./img/{i}.png')
        cnt += 1

In [67]:
def get_info(name, address, real_address, soup):
    cnt = 1
    for i in soup.select('tr td a'):
        i = i.text.strip()
        if cnt % 3 == 1:
            name.append(i)
        elif cnt % 2 == 0:
            address.append(i)
        else:
            real_address.append(i)
        cnt += 1

In [68]:
def get_comment(age, height, weight, crime_date, soup):
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for c in comments:
        if 'td' not in c:
            c.extract()
        else:
            age.append(c.split('나이:')[1][:3].strip())
            height.append(c.split('키:')[1][:3])
            weight.append(c.split('몸무게:')[1][:3].strip())
            date2 = c.split('월')[0][-2:].strip()
            if len(date2) < 2:
                date2 = '0' + date2
            date = c.split('년')[0][-4:] + "-" + date2
            crime_date.append(date)

In [69]:
name_list = []
address_list = []
real_address_list = []
age_list = []
height_list = []
weight_list = []
crime_date = []

for i in range(2, 3):
    if i % 10 == 1:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        get_image(soup)
        get_info(name_list, address_list, real_address_list, soup)
        get_comment(age_list, height_list, weight_list, crime_date, soup)
        driver.find_element_by_link_text(f"다음").click()
    else:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        get_image(soup)
        get_info(name_list, address_list, real_address_list, soup)
        get_comment(age_list, height_list, weight_list, crime_date, soup)
        driver.find_element_by_link_text(f"{i}").click()
    time.sleep(1)

In [70]:
print(name_list)

['선호상', '이창민', '이강희', '김천하', '김현철']


In [385]:
print(weight_list)

['73', '78', '80', '58', '58', '72', '78', '64', '68', '65', '85', '75', '74', '92', '87', '65', '70', '65', '68', '71', '78', '87', '72', '60', '71', '80', '75', '72', '65', '86', '70', '73', '90', '75', '82', '120', '75', '73', '85', '100', '54', '94', '73', '70', '61', '68', '76', '55', '74', '67', '67', '85', '64', '75', '73', '52', '70', '52', '88', '72', '92', '70', '78', '95', '72', '71', '60', '70', '83', '72', '80', '90', '80', '70', '67', '96', '75', '65', '120', '70', '60', '82', '75', '90', '70', '47', '68', '70', '93', '68', '70', '79', '60', '97', '78', '65', '68', '95', '63', '70', '88', '85', '79', '73', '78', '68', '75', '71', '67', '72', '70', '63', '75', '57', '85', '94', '67', '82', '76', '80', '54', '64', '65', '64', '80', '66', '65', '60', '105', '45', '74', '92', '75', '90', '93', '50', '65', '70', '90', '68', '64', '105', '80', '72', '80', '55', '78', '70', '63', '63', '86', '80', '67', '83', '80', '85', '85', '75', '50', '95', '68', '60', '83', '61', '84', '72'

In [75]:
df2 = pd.DataFrame({'name':name_list,
                   'address':address_list,
                   'real_address':real_address_list,
                    'age':age_list,
                   'height':height_list,
                   'weight':weight_list,
                    'crime_date':crime_date
                  })

In [396]:
df.to_csv('trash_list2.csv',encoding='utf-8-sig')

In [88]:
import os
import pandas as pd
import numpy as np
import requests
import folium
import bs4
import json
import base64
from folium import IFrame
from folium import plugins

In [72]:
m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

In [76]:
district_list = []
lat_list = []
long_list = []
api_key = "0C781246-6E25-31B6-966A-E2A614B069E1"

for addr in df2['real_address']: #vworld api를 사용하여 위도 경도 추출
    res = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address={addr}&refine=true&simple=false&format=xml&type=road&key={api_key}")
    soup = bs4.BeautifulSoup(res.content, 'html.parser')
    district_list.append(soup.find('level2').text)
    lat_list.append(soup.find('x').text)
    long_list.append(soup.find('y').text)

In [77]:
df2['district'] = district_list
df2['latitude'] = lat_list
df2['longitude'] = long_list

df2['latitude'] = df2['latitude'].astype('float64')
df2['longitude'] = df2['longitude'].astype('float64')

In [78]:
counts = df2['district'].value_counts()
df2 = df2[~df2['district'].isin(counts[counts < 2].index)]

In [123]:
for lat, long, name in zip(df2['latitude'], df2['longitude'], df2['name']):
    Filename =f'./img/{name}.png'
    encoded = base64.b64encode(open(Filename, 'rb').read())
    html='<img src="data:image/png;base64,{}">'.format #replace 《》with <>
    resolution, width, height = 300, 148, 200
    iframe = IFrame(html(encoded.decode('UTF-8')), width=(width)+20, height=(height)+20)
    popup = folium.Popup(iframe, max_width=1000)
    icon = folium.Icon(color="red", icon="star")
    marker = folium.Marker(location=[long, lat], popup=popup, tooltip = name, icon=icon)
    marker.add_to(m)    
    
#     encoded = base64.b64encode(open(f'{name}.png', 'rb').read())
#     html = '<img src="data:image/png;base64,{}">'.format
#     iframe = IFrame(html(encoded.decode('UTF-8')), width=400, height=350)
#     popup = folium.Popup(iframe, max_width=400)
#     folium.Marker(location=[long, lat], tooltip=html, popup = popup,icon=folium.Icon(color = 'gray')).add_to(map)

In [124]:
df2['name']

1    이창민
2    이강희
Name: name, dtype: object

In [125]:
m